# in this file we do sentiment analysis on the data directly using 3 classfier using tfidf and bow

In [1]:
import pandas as pd
import numpy as np
import json
import re
import string

from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import sys
from sklearn.neighbors import KNeighborsClassifier

import pickle


%matplotlib inline
rand_seed = 0  # random state for reproducibility
np.random.seed(rand_seed)


In [2]:
data = pd.read_excel('data/datasix.xlsx')
data = data.dropna()
data.head()
data.shape
print(data['label'].unique())



['سلبي' 'محايد' 'ايجابي']


In [3]:
data

,comment,label,lemma,clean
0,النت كعب عندنا شديد يا ناس سوداني يعني الزوال...,سلبي,نت كعب عندنا شديد ناسي سوداني عني زوال حول شنا,النت كعب عندنا شديد ناس سوداني يعني الزوال يحو...
1,بس من السحب متين,محايد,سحب متين,السحب متين
2,اول حاجه صلحو خدمتكم سءيه جدا,سلبي,اول حاجه صلح خدمه هو سءيه جد,اول حاجه صلحو خدمتكم سءيه جدا
3,أفضل شبكة علي الإطلاق والشئ البميزكم اكثر واعج...,ايجابي,افضل شبك هو علي اطلاق و بميزم اكثر و اعجبني نت...,افضل شبكه علي الاطلاق والشء البميزكم اكثر واعج...
4,اجهزو لي المقاطعة وشريحتكم دي ح اجيبها ليكم في...,سلبي,اجهز مقاطع هو و شريحه هو دي اجيب هو لي هو مكتب...,اجهزو المقاطعه وشريحتكم دي اجيبها ليكم مكتبكم ...
...,...,...,...,...
6177,الواي فاي كم الاشتراك بتاعو ومميزاتو من الداتا,محايد,واي ف اي اشتراك ب تاعو و مميز داتا,الواي فاي الاشتراك بتاعو وميزاتو الداتا
6178,اقوي شبكه في السودان,ايجابي,اقوي شبكه سودان,اقوي شبكه السودان
6179,اول حاجه غيرو طريقه عملكم لانها اكل مال الناس ...,سلبي,اول حاجه غيرو طريق هو عمل هو لان هو اكل مال ان...,اول حاجه غيرو طريقه عملكم لانها اكل مال الناس ...
6180,سوداني الاقوي والافضل,ايجابي,سوداني اقوي وافل,سوداني الاقوي والافضل


# function to split data into train and test

In [4]:
def random_split(data, features, output, fraction, seed=0):
    X_train, X_test, y_train, y_test = train_test_split(data[features],
                                                        data[output],
                                                        stratify = data[output],
                                                        random_state=seed,
                                                        train_size=fraction
                                                       )
    train_data = pd.DataFrame(data=X_train, columns=features)
    train_data[output] = y_train
    test_data = pd.DataFrame(data=X_test, columns=features)
    test_data[output] = y_test
    
    return train_data, test_data

# here we split data as :80% train ,20% test

In [5]:
train_fraction = .80
output = 'label' 
features = data.columns.tolist() 
features.remove(output)
print('output:', output)
print('features:', features)

train_data, test_data = random_split(data, features, output, train_fraction, rand_seed)

print("train data =")
print(len(train_data))
print(len(test_data))
print(len(data))

print(data['label'].unique())
print(train_data['label'].unique())
print(test_data['label'].unique())



print("train data =")
print(len(train_data))


print(len(train_data['label']=='سلبي'))
print(len(train_data['label']=='ايجابي'))
print(len(train_data['label']=='محايد'))

print("test data =")
print(len(test_data))

print(len(test_data['label']=='سلبي'))
print(len(test_data['label']=='ايجابي'))
print(len(test_data['label']=='محايد'))

print("all data =")
print(len(data))

print(len(data['label']=='سلبي'))
print(len(data['label']=='ايجابي'))
print(len(data['label']=='محايد'))

output: label
features: ['comment', 'lemma', 'clean']
train data =
4928
1233
6161
['سلبي' 'محايد' 'ايجابي']
['محايد' 'ايجابي' 'سلبي']
['محايد' 'سلبي' 'ايجابي']
train data =
4928
4928
4928
4928
test data =
1233
1233
1233
1233
all data =
6161
6161
6161
6161


In [6]:
print(" train ")
trainP=train_data[train_data['label'] == 'محايد']
trainNE=train_data[train_data['label'] == 'سلبي']
trainNg=train_data[train_data['label'] == 'ايجابي']

print(len(trainP),len(trainNE),len(trainNg))

 train 
1085 2438 1405


In [7]:
testP=test_data[test_data['label'] == 'محايد']
testNE=test_data[test_data['label'] == 'سلبي']
testNg=test_data[test_data['label'] == 'ايجابي']

print(len(testP),len(testNE),len(testNg))

272 610 351


In [8]:
pP=data[data['label'] == 'محايد']
pNE=data[data['label'] == 'سلبي']
pNg=data[data['label'] == 'ايجابي']

print(len(pP),len(pNE),len(pNg))

1357 3048 1756


# using tf idf to extract features

In [9]:

# TF IDF


vectorizer = TfidfVectorizer(ngram_range=(1, 2), sublinear_tf=True, max_df=0.5, stop_words=None, use_idf=True)
train_data_features = vectorizer.fit_transform(train_data['comment'])
test_data_features = vectorizer.transform(test_data['comment'])

#SHAPE

train_data_features.shape, test_data_features.shape

#FUNCTION FOR MODEL TRAIN



((4928, 37313), (1233, 37313))

# function used to train the model and make a report for it,s score

#### def train_n_test_classifier(clf, train_features, train_labels, test_features, test_labels,data):
    clf.fit(train_features, train_labels) # please learn patterns from the data

   
    print('-'*100+str(clf)[0:15])
    print("accuracy_score Score on training data:")
    print(clf.score(train_features, train_labels))
    
    
    print('_'*100)

    print("score on testing data:")
    
    pred_y = clf.predict(test_features)
    
    data['predict']=pred_y
 
    count=0
    
    print("accuracy_score Score on test data:")
    print(accuracy_score(test_labels, pred_y))
    
    print("f1_score  on test data:")
    print(f1_score(test_labels, pred_y, average='macro'))
    filename='result_'+str(clf)[0:15]+'.xlsx'
    data.to_excel(filename)
    
    
    sentiment_fit=clf.fit(train_features,train_labels)
    y_pred=sentiment_fit.predict(test_features)
    print(classification_report(test_labels,y_pred,target_names=('posative','negative','netural')))


In [12]:
def train_n_test_classifier(clf, train_features, train_labels, test_features, test_labels,data):
    clf.fit(train_features, train_labels) # please learn patterns from the data


    print('-'*100+str(clf)[0:15])
    print("accuracy_score Score on training data:")
    print(clf.score(train_features, train_labels))


    print('_'*100)

    print("score on testing data:")

    pred_y = clf.predict(test_features)

    data['predict']=pred_y

    count=0

    print("accuracy_score Score on test data:")
    print(accuracy_score(test_labels, pred_y))

    print("f1_score  on test data:")
    print(f1_score(test_labels, pred_y, average='macro'))
    filename='result_'+str(clf)[0:15]+'.xlsx'
    data.to_excel(filename)


    sentiment_fit=clf.fit(train_features,train_labels)
    y_pred=sentiment_fit.predict(test_features)
    print(classification_report(test_labels,y_pred,target_names=('posative','negative','netural')))

# pass the models to the train function

In [13]:
#LOGASTIC REGRESSION



logistic_reg = LogisticRegression(random_state=rand_seed)

train_n_test_classifier(logistic_reg, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)



# MultinomialNB

mnb = MultinomialNB()

train_n_test_classifier(mnb,  train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)
# S V M

svm = SVC(kernel='linear', probability=True, random_state=rand_seed)

train_n_test_classifier(svm,  train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)


# MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(20,20,20,20), verbose=True, tol=0.001, random_state=rand_seed)
train_n_test_classifier(mlp,  train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)

# KNN

knn = KNeighborsClassifier(n_neighbors=3)

train_n_test_classifier(knn, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)

#TREE
dtree = DecisionTreeClassifier(random_state=0)
train_n_test_classifier(dtree, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)


----------------------------------------------------------------------------------------------------LogisticRegress
accuracy_score Score on training data:
0.9281655844155844
____________________________________________________________________________________________________
score on testing data:
accuracy_score Score on test data:
0.7575020275750203
f1_score  on test data:
0.7040118888436219
              precision    recall  f1-score   support

    posative       0.88      0.74      0.81       351
    negative       0.71      0.94      0.81       610
     netural       0.74      0.37      0.49       272

    accuracy                           0.76      1233
   macro avg       0.78      0.68      0.70      1233
weighted avg       0.77      0.76      0.74      1233

----------------------------------------------------------------------------------------------------MultinomialNB()
accuracy_score Score on training data:
0.8871753246753247
__________________________________________________

In [12]:
estimator = []


estimator.append(('LR', LogisticRegression(solver ='lbfgs',  multi_class ='multinomial',  max_iter = 200)))
estimator.append(('SVC', SVC(gamma ='auto', probability = True)))
estimator.append(('DTC', DecisionTreeClassifier()))
estimator.append(('mnb',MultinomialNB()))
estimator.append(('mlp',MLPClassifier(hidden_layer_sizes=(20,20,20,20), verbose=True, tol=0.001, random_state=rand_seed)))
estimator.append(('knn',KNeighborsClassifier(n_neighbors=3)))     


In [13]:

vot_hard = VotingClassifier(estimators = estimator, voting ='hard')

train_n_test_classifier(vot_hard, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)



Iteration 1, loss = 1.04343732
Iteration 2, loss = 1.00424941
Iteration 3, loss = 0.87860944
Iteration 4, loss = 0.67202608
Iteration 5, loss = 0.53564571
Iteration 6, loss = 0.43555311
Iteration 7, loss = 0.31544418
Iteration 8, loss = 0.19940831
Iteration 9, loss = 0.10940349
Iteration 10, loss = 0.06382355
Iteration 11, loss = 0.04676391
Iteration 12, loss = 0.03802096
Iteration 13, loss = 0.03509686
Iteration 14, loss = 0.03314875
Iteration 15, loss = 0.02949797
Iteration 16, loss = 0.02839154
Iteration 17, loss = 0.02591615
Iteration 18, loss = 0.02650623
Iteration 19, loss = 0.02497732
Iteration 20, loss = 0.02488126
Iteration 21, loss = 0.02408145
Iteration 22, loss = 0.02272539
Iteration 23, loss = 0.02253886
Iteration 24, loss = 0.02348083
Iteration 25, loss = 0.02314594
Iteration 26, loss = 0.02263603
Iteration 27, loss = 0.02138417
Iteration 28, loss = 0.02065151
Iteration 29, loss = 0.02066999
Iteration 30, loss = 0.02139000
Iteration 31, loss = 0.02160202
Iteration 32, los

In [14]:

vot_soft = VotingClassifier(estimators = estimator, voting ='soft')

train_n_test_classifier(vot_soft, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)



Iteration 1, loss = 1.04343732
Iteration 2, loss = 1.00424941
Iteration 3, loss = 0.87860944
Iteration 4, loss = 0.67202608
Iteration 5, loss = 0.53564571
Iteration 6, loss = 0.43555311
Iteration 7, loss = 0.31544418
Iteration 8, loss = 0.19940831
Iteration 9, loss = 0.10940349
Iteration 10, loss = 0.06382355
Iteration 11, loss = 0.04676391
Iteration 12, loss = 0.03802096
Iteration 13, loss = 0.03509686
Iteration 14, loss = 0.03314875
Iteration 15, loss = 0.02949797
Iteration 16, loss = 0.02839154
Iteration 17, loss = 0.02591615
Iteration 18, loss = 0.02650623
Iteration 19, loss = 0.02497732
Iteration 20, loss = 0.02488126
Iteration 21, loss = 0.02408145
Iteration 22, loss = 0.02272539
Iteration 23, loss = 0.02253886
Iteration 24, loss = 0.02348083
Iteration 25, loss = 0.02314594
Iteration 26, loss = 0.02263603
Iteration 27, loss = 0.02138417
Iteration 28, loss = 0.02065151
Iteration 29, loss = 0.02066999
Iteration 30, loss = 0.02139000
Iteration 31, loss = 0.02160202
Iteration 32, los

In [15]:

Gradientclf = GradientBoostingClassifier(n_estimators=100,learning_rate=1.0,max_depth=1, random_state=0)

train_n_test_classifier(Gradientclf,train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)

adaClf = AdaBoostClassifier(n_estimators=100, random_state=0)

train_n_test_classifier(adaClf, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)

rf = RandomForestClassifier(n_estimators=100, random_state=rand_seed)

train_n_test_classifier(rf, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)


----------------------------------------------------------------------------------------------------GradientBoostin
accuracy_score Score on training data:
0.8224431818181818
____________________________________________________________________________________________________
score on testing data:
accuracy_score Score on test data:
0.6845093268450932
f1_score  on test data:
0.6478328324822843
              precision    recall  f1-score   support

    posative       0.75      0.68      0.72       351
    negative       0.70      0.80      0.74       610
     netural       0.55      0.43      0.49       272

    accuracy                           0.68      1233
   macro avg       0.67      0.64      0.65      1233
weighted avg       0.68      0.68      0.68      1233

----------------------------------------------------------------------------------------------------AdaBoostClassif
accuracy_score Score on training data:
0.7173295454545454
__________________________________________________

# use BOW 

In [16]:
vectorizer = CountVectorizer(ngram_range=(1, 2))

# bow


vectorizer = CountVectorizer(ngram_range=(1, 2))
train_data_features = vectorizer.fit_transform(train_data['comment'])
test_data_features = vectorizer.transform(test_data['comment'])

#SHAPE

train_data_features.shape,  test_data_features.shape


((4928, 37313), (1233, 37313))

In [18]:
#LOGASTIC REGRESSION



logistic_reg = LogisticRegression(random_state=rand_seed)

train_n_test_classifier(logistic_reg, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)



# MultinomialNB

mnb = MultinomialNB()

train_n_test_classifier(mnb,  train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)
# S V M

svm = SVC(kernel='linear', probability=True, random_state=rand_seed)

train_n_test_classifier(svm,  train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)


# MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(20,20,20,20), verbose=True, tol=0.001, random_state=rand_seed)
train_n_test_classifier(mlp,  train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)

# KNN

knn = KNeighborsClassifier(n_neighbors=3)

train_n_test_classifier(knn, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)

#TREE
dtree = DecisionTreeClassifier(random_state=0)
train_n_test_classifier(dtree, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)


----------------------------------------------------------------------------------------------------LogisticRegress
accuracy_score Score on training data:
0.9876217532467533
____________________________________________________________________________________________________
score on testing data:
accuracy_score Score on test data:
0.7639902676399026
f1_score  on test data:
0.727419714599249
              precision    recall  f1-score   support

    posative       0.82      0.78      0.80       351
    negative       0.77      0.87      0.82       610
     netural       0.66      0.49      0.56       272

    accuracy                           0.76      1233
   macro avg       0.75      0.72      0.73      1233
weighted avg       0.76      0.76      0.76      1233

----------------------------------------------------------------------------------------------------MultinomialNB()
accuracy_score Score on training data:
0.9563717532467533
___________________________________________________

In [19]:
estimator = []


estimator.append(('LR', LogisticRegression(solver ='lbfgs',  multi_class ='multinomial',  max_iter = 200)))
estimator.append(('SVC', SVC(gamma ='auto', probability = True)))
estimator.append(('DTC', DecisionTreeClassifier()))
estimator.append(('mnb',MultinomialNB()))
estimator.append(('mlp',MLPClassifier(hidden_layer_sizes=(20,20,20,20), verbose=True, tol=0.001, random_state=rand_seed)))
estimator.append(('knn',KNeighborsClassifier(n_neighbors=3)))     


In [20]:

vot_hard = VotingClassifier(estimators = estimator, voting ='hard')

train_n_test_classifier(vot_hard, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)


vot_soft = VotingClassifier(estimators = estimator, voting ='soft')

train_n_test_classifier(vot_soft, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)


Gradientclf = GradientBoostingClassifier(n_estimators=100,learning_rate=1.0,max_depth=1, random_state=0)

train_n_test_classifier(Gradientclf,train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)

adaClf = AdaBoostClassifier(n_estimators=100, random_state=0)

train_n_test_classifier(adaClf, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)

rf = RandomForestClassifier(n_estimators=100, random_state=rand_seed)

train_n_test_classifier(rf, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)


Iteration 1, loss = 1.03293933
Iteration 2, loss = 0.91646091
Iteration 3, loss = 0.68796537
Iteration 4, loss = 0.50769331
Iteration 5, loss = 0.35054071
Iteration 6, loss = 0.19965777
Iteration 7, loss = 0.09508730
Iteration 8, loss = 0.05500684
Iteration 9, loss = 0.04372054
Iteration 10, loss = 0.03554988
Iteration 11, loss = 0.03288744
Iteration 12, loss = 0.02770656
Iteration 13, loss = 0.02759942
Iteration 14, loss = 0.02623975
Iteration 15, loss = 0.02424020
Iteration 16, loss = 0.02354085
Iteration 17, loss = 0.02196322
Iteration 18, loss = 0.02282241
Iteration 19, loss = 0.02243948
Iteration 20, loss = 0.02159874
Iteration 21, loss = 0.02268327
Iteration 22, loss = 0.02089251
Iteration 23, loss = 0.02143801
Iteration 24, loss = 0.02231997
Iteration 25, loss = 0.02189820
Iteration 26, loss = 0.02098324
Iteration 27, loss = 0.02045528
Iteration 28, loss = 0.02066248
Training loss did not improve more than tol=0.001000 for 10 consecutive epochs. Stopping.
-----------------------